In [1]:
import tensorflow as tf

In [19]:
z = tf.placeholder(tf.float32, (None, 100))
training = tf.placeholder(tf.bool)
alpha = 0.2

In [21]:
leaky_relu = lambda x : tf.maximum(alpha * x, x)

de_conv = lambda x, flt : tf.layers.conv2d_transpose(x, filters = flt, kernel_size= (5, 5), 
                                                       strides=(2, 2), padding = 'same', 
                                                       use_bias=False, activation=None)
norm_and_relu = lambda x : leaky_relu(tf.layers.batch_normalization(x, training=training))

fc = tf.layers.dense(z, 4 * 4* 512, activation=None, use_bias=False)
fc = tf.reshape(fc, (-1, 4, 4, 512))
fc = norm_and_relu(fc)
print(fc)

c1 = norm_and_relu(de_conv(fc, 256))
print(c1)

c2 = norm_and_relu(de_conv(c1, 128))
print(c2)

# Output layer, 32x32x3
logits = de_conv(c2, 3)

out = tf.tanh(logits)
print(out)

Tensor("Maximum_3:0", shape=(?, 4, 4, 512), dtype=float32)
Tensor("Maximum_4:0", shape=(?, 8, 8, 256), dtype=float32)
Tensor("Maximum_5:0", shape=(?, 16, 16, 128), dtype=float32)
Tensor("Tanh_1:0", shape=(?, 32, 32, 3), dtype=float32)


In [28]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
training = tf.placeholder(tf.bool)
alpha = 0.2


leaky_relu = lambda x : tf.maximum(alpha * x, x)
norm_and_leaky_relu = lambda x : leaky_relu(tf.layers.batch_normalization(x, training=training))
conv = lambda x, flt : tf.layers.conv2d(x, filters = flt, kernel_size= (3, 3), 
                                                               strides=(2, 2), padding = 'same', 
                                                               use_bias=True, activation=None) 

conv1 = conv(x, 16)
conv1 = norm_and_leaky_relu(conv1)
print(conv1)

conv2 = conv(conv1, 32)
conv2 = norm_and_leaky_relu(conv2)
print(conv2)

conv3 = conv(conv2, 64)
conv3 = norm_and_leaky_relu(conv3)
print(conv3)

conv3_flat = tf.reshape(conv3, (-1, 4 * 4 * 64))
fc = tf.layers.dense(conv3_flat, 1)
print(fc)

Tensor("Maximum_21:0", shape=(?, 16, 16, 16), dtype=float32)
Tensor("Maximum_22:0", shape=(?, 8, 8, 32), dtype=float32)
Tensor("Maximum_23:0", shape=(?, 4, 4, 64), dtype=float32)
Tensor("dense_20/BiasAdd:0", shape=(?, 1), dtype=float32)


In [31]:
def discriminator(x, reuse=False, alpha=0.2):
    with tf.variable_scope('discriminator', reuse=reuse):
        # Input layer is 32x32x3
        
        """
        MW Notes:
        per the solution video, didnt use batch norm on the first layer
        per the solution video used a deeper model (64, 128, 256) and left kernel at size 5, 5
        """
        print(training)
        
        leaky_relu = lambda x : tf.maximum(alpha * x, x)
        norm_and_leaky_relu = lambda x : leaky_relu(tf.layers.batch_normalization(x, training=training))
        conv = lambda x, flt : tf.layers.conv2d(x, filters = flt, kernel_size= (5, 5), 
                                                                       strides=(2, 2), padding = 'same', 
                                                                       use_bias=True, activation=None) 
        
        conv1 = conv(x, 64)
        conv1 = leaky_relu(conv1)
        print(conv1)

        conv2 = conv(conv1, 128)
        conv2 = norm_and_leaky_relu(conv2)
        print(conv2)

        conv3 = conv(conv2, 256)
        conv3 = norm_and_leaky_relu(conv3)
        print(conv3)

        conv3_flat = tf.reshape(conv3, (-1, 4 * 4 * 256))
        
        logits = tf.layers.dense(conv3_flat, units = 1, activation=None)
        out = tf.nn.sigmoid(logits)
        
        return out, logits

In [34]:
tf.reset_default_graph()
discriminator(x)

Tensor("Placeholder_18:0", dtype=bool)


ValueError: Tensor("discriminator/conv2d/kernel:0", shape=(5, 5, 3, 64), dtype=float32_ref) must be from the same graph as Tensor("Placeholder_17:0", shape=(?, 32, 32, 3), dtype=float32).